<a href="https://colab.research.google.com/github/ARJUN108-verma/Demo/blob/main/tune_hyperparameter_with_the_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Tune hyperparameter with the keras tuner")

Tune hyperparameter with the keras tuner


Setup

In [2]:
import tensorflow as tf
from tensorflow import keras

install and import keras tuner

In [4]:
pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [5]:
import keras_tuner as kt

Dwonload and Prepare the dataset

In [6]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


normalize the pixel value between 0 and 1

In [7]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

Define the model

In [8]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape = (28,28)))

  #tune the number of units in first dense layer
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  #tune the learning rate for optimizer
  #choose an optimal value from 0.01 ,0.001 or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compiler(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                 loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics = ['accuracy'])
  return model

Instantiate the tuner and perform hypertuning

In [10]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras

def model_builder(hp):
    model = keras.Sequential([
        keras.layers.Input(shape=(28, 28)),  # Explicit Input layer
        keras.layers.Flatten(),
        keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        keras.layers.Dense(10)
    ])

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Fix the typo here
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

# Create a tuner object
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)


In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [12]:
tuner.search(img_train, label_train, epochs = 50, validation_split= 0.2, callbacks = [stop_early])

#get the optimal hyperparameter
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparanmeter search is complete .The optimal number of units in first densly-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optiminzer is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 53s]
val_accuracy: 0.8582500219345093

Best val_accuracy So Far: 0.893833339214325
Total elapsed time: 00h 19m 58s

The hyperparanmeter search is complete .The optimal number of units in first densly-connected
layer is 448 and the optimal learning rate for the optiminzer is 0.001.



Train the model

In [14]:
# Build the best model from tuner
model = tuner.hypermodel.build(best_hps)

# Train the model (fixing the typo in 'epochs')
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

# Get validation accuracy per epoch
val_acc_per_epoch = history.history['val_accuracy']


Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.7819 - loss: 0.6164 - val_accuracy: 0.8538 - val_loss: 0.4033
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8671 - loss: 0.3699 - val_accuracy: 0.8733 - val_loss: 0.3538
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8768 - loss: 0.3384 - val_accuracy: 0.8785 - val_loss: 0.3401
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8852 - loss: 0.3075 - val_accuracy: 0.8735 - val_loss: 0.3618
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8944 - loss: 0.2850 - val_accuracy: 0.8844 - val_loss: 0.3318
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9002 - loss: 0.2720 - val_accuracy: 0.8846 - val_loss: 0.3296
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9050 - loss: 0.2515 - val_accuracy: 0.8867 - val_loss: 0.3231
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.9086 - loss: 0.246

In [15]:
#build the model
model = tuner.hypermodel.build(best_hps)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 35


In [16]:
hypermodel = tuner.hypermodel.build(best_hps)

#retrain the model
hypermodel.fit(img_train, label_train, epochs = best_epoch, validation_split= 0.2)

Epoch 1/35
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.7832 - loss: 0.6144 - val_accuracy: 0.8303 - val_loss: 0.4550
Epoch 2/35
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.8618 - loss: 0.3798 - val_accuracy: 0.8654 - val_loss: 0.3630
Epoch 3/35
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8785 - loss: 0.3372 - val_accuracy: 0.8637 - val_loss: 0.3692
Epoch 4/35
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8886 - loss: 0.3040 - val_accuracy: 0.8679 - val_loss: 0.3545
Epoch 5/35
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8931 - loss: 0.2837 - val_accuracy: 0.8877 - val_loss: 0.3196
Epoch 6/35
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.8998 - loss: 0.2696 - val_accuracy: 0.8773 - val_loss: 0.3410
Epoch 7/35
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.9030 - loss: 0.2573 - val_accuracy: 0.8853 - val_loss: 0.3242
Epoch 8/35
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9117 - loss: 0.2

to finish this tutorial , evaluate the hypermodel on test data

In [17]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8880 - loss: 0.4972
[test loss, test accuracy]: [0.48408710956573486, 0.8885999917984009]
